<a href="https://colab.research.google.com/github/AdedokunOluremi/Coursera_Capstone/blob/master/Capstone_Project_Segmenting_and_Clustering_Lagos_State_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploring, Segmenting and Clustering Lagos City Nigeria using Geolocation Data.

## Oluremi Atilade Adedokun

January 13, 2021

## **1. Introduction**

1.1 **Background**<br>
I lived in Lagos for four years. I worked with Flour Mills of Nigeria Plc. as a Milling Technologist. At that time, the company is the second largest flour milling company in the world and has more than ten fully automated Buhler mills and a daily capacity of more than 8000tons.
Lagos, sometimes referred to as Lagos State to distinguish it from Lagos Metropolitan Area, is a state located in southwestern Nigeria. The smallest in area of Nigeria's 36 states, Lagos State is arguably the most economically important state of the country, containing Lagos, the nation's largest urban area. It is a major financial centre and would be the fifth largest economy in Africa, if it were a country.

It has the highest population density of Nigeria's states. The actual population total is disputed between the official Nigerian Census of 2006 and a much higher figure claimed by the Lagos State Government. Lagos State annual GDP is 1 trillion naira.

Lagos State is bounded on the north and east by Ogun State. In the west it shares boundaries with the Republic of Benin. Behind its southern borders lies the Atlantic Ocean. 22% of its 3,577 km2 are lagoons and creeks.

Victoria Island, the financial center of the metropolis, is known for its beach resorts, boutiques and nightlife. To the north, Lagos Island is home to the National Museum Lagos, displaying cultural artifacts and craftworks. Nearby is Freedom Park, once a colonial-era prison, now a major venue for concerts and public events. https://en.wikipedia.org/wiki/Lagos  ([link](https://en.wikipedia.org/wiki/Lagos))<br>
I want to use this opportunity to help future startups and restaurants find the most ideal location in Lagos city for their business.<br>
<br>


1.2 **Problem**<br>
The problem I want to solve is using Geo-location Data to explore, segment and cluster neighborhoods in Lagos Nigeria to help a Tech-StartUp find the ideal location for its office in The City of Lagos Nigeria and also help a restaurant find an ideal location to open it's business.<br>
<br>
1.3 **Interest**<br>
Obviously, a contractor looking to open a new Tech-StartUp in Lagos would be very interested in accurate prediction of appropriate location
for competitive advantage and business values. Also someone who is interested in opening a new restaurant will also be interested.

# **2.** **Data acquisition and cleaning**<br>


2.1 **Data sources**<br>
**The Dataset is the wikipedia page of Lagos state** see [link](https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population)<br>
<br>
2.2 **Data Cleaning**<br>
We shall scrape the web page using the beautiful soup library
and use The Foursquare API calls to retrieve geolocation data,
We shall fetch the text data using the requests library,
and convert it from JSON to Pandas data frame using the json_normalize module
We shall use The folium library to render the maps and plot these via The Matplotlib library. Then we shall explore respective boroughs and analyse each area as a location for a Tech-Startup based on the aforementioned parameters.
After this, we shall select our top location.<br>
<br>
2.3  **Feature selection**<br>
Upon completing the analysis, we shall perform feature selection.
